In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [2]:
#nltk.download('punkt')
#nltk.download('wordnet')

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['Thank', 'you', 'so', 'much'], 'thanks'), ([], 'noanswer'), (['How', 'could', 'you', 'help', 'me', '?'], 'purpose'), (['What', 'you', 'can', 'do', '?'], 'purpose'), (['What', 'help', 'you', 'provide', '?'], 'purpose'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'purpose'), (['What', 'support', 'is', 'offered'], 'purpose'), (['Do', 'i', 'have', 'a', 'meeting', 'today', '?'], 'schedule'), (['What', 'are', 'my'

In [5]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

29 documents
6 classes ['goodbye', 'greeting', 'noanswer', 'purpose', 'schedule', 'thanks']
56 unique lemmatized words ["'s", ',', 'a', 'am', 'anyone', 'anything', 'are', 'awesome', 'be', 'busy', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'me', 'meeting', 'much', 'must', 'my', 'next', 'nice', 'offered', 'plan', 'provide', 'see', 'should', 'so', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'today', 'what', 'you']


In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # output is a '0' for each tag and '1' for current tag (for each pattern)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print(training)
print("Training data created")

[[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
  list([0, 0, 0, 1, 0, 0])]
 [list([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
  list([0, 0, 0, 0, 1, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
  list([1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0])
  list([0, 0, 0, 0, 1, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  l

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
6/6 [==============================] - 0s 6ms/step - loss: 1.8805 - accuracy: 0.1034
Epoch 2/200
6/6 [==============================] - 0s 833us/step - loss: 1.7501 - accuracy: 0.2759
Epoch 3/200
6/6 [==============================] - 0s 1ms/step - loss: 1.6967 - accuracy: 0.2414
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 1.6046 - accuracy: 0.4828
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6535 - accuracy: 0.2759
Epoch 6/200
6/6 [==============================] - 0s 1ms/step - loss: 1.5659 - accuracy: 0.4483
Epoch 7/200
6/6 [==============================] - 0s 1ms/step - loss: 1.3350 - accuracy: 0.6552
Epoch 8/200
6/6 [==============================] - 0s 1ms/step - loss: 1.2959 - accuracy: 0.6897
Epoch 9/200
6/6 [==============================] - 0s 997us/step - loss: 1.2427 - accuracy: 0.6207
Epoch 10/200
6/6 [==============================] - 0s 1ms/step - loss: 1.1049 - accuracy: 0.6207
Epoch 11/200
6/6 [=======

6/6 [==============================] - 0s 1ms/step - loss: 0.0425 - accuracy: 1.0000
Epoch 85/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0474 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1361 - accuracy: 0.9655
Epoch 87/200
6/6 [==============================] - 0s 831us/step - loss: 0.0406 - accuracy: 1.0000
Epoch 88/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0581 - accuracy: 0.9655
Epoch 89/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0461 - accuracy: 1.0000
Epoch 90/200
6/6 [==============================] - 0s 997us/step - loss: 0.1182 - accuracy: 0.9655
Epoch 91/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1581 - accuracy: 0.9655
Epoch 92/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1212 - accuracy: 0.9655
Epoch 93/200
6/6 [==============================] - 0s 997us/step - loss: 0.0721 - accuracy: 0.9655
Epoch 94/200
6/6 [=========

6/6 [==============================] - 0s 1ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 167/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0147 - accuracy: 1.0000
Epoch 168/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0547 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 997us/step - loss: 0.0392 - accuracy: 1.0000
Epoch 170/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0177 - accuracy: 1.0000
Epoch 171/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0299 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 997us/step - loss: 0.0443 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 3ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 174/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0173 - accuracy: 1.0000
Epoch 175/200
6/6 [==============================] - 0s 997us/step - loss: 0.0169 - accuracy: 1.0000
Epoch 176/200
6/6 